<a href="https://colab.research.google.com/github/NikosKats/ColabFiles/blob/CIFAR10-CNN-With-Data-Changes.ipynb/CIFAR10_CNN_With_Data_Changes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Our second experiment trains the model in the original CIFAR-10 dataset but this time with changes to the dataset by applying horizontal flipping and random cropping to the data with applied padding. Below the code is implemented and explained step by step through the process.


First, you will need to import the necessary libraries and set some parameters for the training process:


In [10]:
import torch
import torchvision
import torchvision.transforms as transforms

batch_size = 128
num_epochs = 10
learning_rate = 0.001


use the torchvision.transforms library to apply random horizontal flipping and random cropping to the training images:

In [11]:
# Define transforms for the training dataset
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor()
])

Next, you will need to load the CIFAR-10 dataset and apply any necessary preprocessing:

In [12]:
# Load the CIFAR-10 dataset with the defined transforms
train_dataset = torchvision.datasets.CIFAR10(root='path/to/data', train=True,
                                        download=True, transform=train_transform)

test_dataset = torchvision.datasets.CIFAR10(root='path/to/data', train=False,
                                       download=True, transform=transforms.ToTensor())

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


Then you will need to define your model, in this case a convolutional neural network:

In [13]:
import torch.nn as nn

# Define the model
class CIFAR10Model(nn.Module):
    def __init__(self):
        super(CIFAR10Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = self.fc1(x)
        return x

# Create an instance of the model
model = CIFAR10Model()

Then you will need to define a loss function and an optimizer:

In [14]:
# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Now you can train your model:

In [15]:
# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Print the current loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 1.3393
Epoch [2/10], Loss: 1.1556
Epoch [3/10], Loss: 1.1895
Epoch [4/10], Loss: 1.1708
Epoch [5/10], Loss: 1.0087
Epoch [6/10], Loss: 1.0044
Epoch [7/10], Loss: 1.0286
Epoch [8/10], Loss: 1.0977
Epoch [9/10], Loss: 0.9385
Epoch [10/10], Loss: 1.0533


Once the model is trained, you can evaluate its performance on the test dataset:

In [16]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f'Accuracy of the model on the test images: {100 * correct / total}%')

Accuracy of the model on the test images: 68.93%


If we apply the same training in the same dataset but the second time directly to the distorted data we see that the accuracy of the model has 2.37% less accurate results.

Achieving an accuracy of 99% on the CIFAR-10 dataset is a challenging task and may require a combination of several techniques. Here are a few suggestions that may help to improve the performance of your model:

1. Use a deeper and more complex architecture: A deeper and more complex architecture, such as a ResNet or a DenseNet, can provide more capacity to the model to learn features from the dataset.

2. Use pre-trained models: You can use pre-trained models on large datasets like ImageNet, and fine-tune them on your dataset. This can help to leverage the features learned by the pre-trained model and improve the performance on the task.

3. Use more advanced data augmentation techniques: You can try more advanced data augmentation techniques such as Cutout, Mixup, and Dropout. These techniques can help to increase the diversity of the training dataset and improve the generalization performance of the model.

4. Use a more sophisticated optimizer: You can try more advanced optimization algorithms such as RMSprop, Adagrad, or AdamW which can help to improve the convergence of the model.

5. Use ensemble methods: Ensemble methods involve training multiple models and combining their predictions. These methods can help to reduce the variance of the model and improve its performance.

6. Hyperparameter tuning: Hyperparameter tuning is the process of fine-tuning the hyperparameters of a model to find the best configuration. You can use techniques such as Grid Search, Random Search, or Bayesian optimization to find the optimal hyperparameters.

7. Regularization: Regularization is a technique used to prevent overfitting by adding a penalty term to the loss function. Regularization techniques such as dropout, weight decay, and early stopping can help to reduce overfitting and improve the generalization performance of the model.

Keep in mind that achieving an accuracy of 99% on the CIFAR-10 dataset is a challenging task and may require a lot of experimentation and fine-tuning. It's also important to note that, sometimes, even with all these techniques, the accuracy may not reach 99%. But with the combination of these techniques, the accuracy will be close to 99%

Finally, I want to remind you that training a model with a high accuracy is not the only goal, you may want to consider other factors such as computational cost, interpretability, and generalization capabilities of the model.

Cutout: Cutout is a regularization technique that randomly masks out a portion of the input images. The following code applies Cutout to the training images with a patch size of 16x16 pixels:

In [ ]:
from albumentations import Cutout

# Define the Cutout transform
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    Cutout(num_holes=1, max_h_size=16, max_w_size=16, always_apply=False, p=0.5),
    transforms.ToTensor()
])


Mixup: Mixup is a regularization technique that generates new training examples by interpolating between two randomly chosen examples. The following code applies Mixup to the training images:

In [ ]:
from albumentations.augmentations import functional as F
from albumentations import Mixup

def mixup_transform(alpha, transform=None):
    def augment(data):
        data = transform(**data) if transform else data
        img, label = data
        img1, img2, label1, label2 = F.mixup(img, label, alpha=alpha)
        return img1, label1, img2, label2
    return augment

# Define the Mixup transform
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    Mixup(alpha=1.0, transform=None),
    transforms.ToTensor()
])


Dropout: Dropout is a regularization technique that randomly sets a portion of the input units to zero during training. The following code applies Dropout to the training images with a drop probability of 0.2:



In [ ]:
from albumentations import Dropout

# Define the Dropout transform
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    Dropout(p=0.2, always_apply=False),
    transforms.ToTensor()
])


You should also keep in mind that, when using these advanced data augmentation techniques, you may need to adjust the hyperparameters and the architecture of the model accordingly to achieve the best performance. You will need to experiment and fine-tune the model to achieve the best performance.